## 这里测试根据文档内容构建embedding数据并保存到本地json文件中

### 这里只是为了以后不要对大文档重复调用openai接口生成embeddings，烧钱...

In [1]:
PINECONE_API_KEY = '19156849-2ed3-4cd6-80b4-a05964d4e264'
PINECONE_API_ENV = 'eu-west1-gcp'
index_name = "hello2"

In [2]:
import json
# 从JSON文件中加载embeddings数据
loaded_data = []
with open('embeddings.json', 'r') as f:
    loaded_data = json.load(f)

# 打印数据类型
print(type(loaded_data))  # <class 'dict'>

<class 'dict'>


In [3]:
# extract embeddings to a list
embeds = [record['embedding'] for record in loaded_data['data']]
print(len(embeds))
content_list = [record['content'] for record in loaded_data['data']]
print(len(content_list))

6
6


In [4]:
# 这里输出向量维度，需要在pinecone中配置相同的dimensions数值
print(len(embeds[0]))

1536


In [5]:
import pinecone

C:\Users\15271\AppData\Local\Programs\Python\Python311\Lib\site-packages\pinecone\index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [6]:
# initialize pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_API_ENV  # next to api key in console
)

### 查看当前pinecone中index的描述信息

In [7]:
index_description = pinecone.describe_index(index_name)
index_description

IndexDescription(name='hello2', metric='cosine', replicas=1, dimension=1536.0, shards=1, pods=1, pod_type='s1.x1', status={'ready': True, 'state': 'Ready'}, metadata_config=None, source_collection='')

In [8]:
index = pinecone.Index(index_name)

In [9]:
# 这里需要自己定义每一个向量的id和meta数据（meta数据后续可以直接存储原文，这里暂时没做）
ids_batch = [f'id_{i}' for i in range(0, len(embeds)+1)]
metas_batch = [{'text': content_list[i]} for i in range(0, len(content_list))]
to_upsert = zip(ids_batch, embeds, metas_batch)
# upsert to Pinecone
index.upsert(vectors=list(to_upsert))

{'upserted_count': 6}

In [10]:
index_stats = index.describe_index_stats()
print(index_stats)

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 6}},
 'total_vector_count': 6}


In [11]:
fetch_response = index.fetch(ids=['id_0', 'id_1'])

In [ ]:
fetch_response